<a href="https://colab.research.google.com/github/junsu122/AI_Basic/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EB%94%A5%EB%9F%AC%EB%8B%9D_%EA%B8%B0%EC%B4%88/%EC%8B%A0%EA%B2%BD%EB%A7%9D/MSE_BCEWithLogits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 공통 신경망 구조 (BatchNorm 포함)
class ComparisonNet(nn.Module):
    def __init__(self, output_size):
        super(ComparisonNet, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(10, 20),
            nn.BatchNorm1d(20),  # [개념: BatchNorm] 데이터 분포 정규화
            nn.ReLU(),
            nn.Linear(20, output_size)
        )

    def forward(self, x):
        return self.layer(x)

# 가상 데이터 생성 (100개의 샘플, 10개의 특징)
inputs = torch.randn(100, 10)
# 회귀용 정답 (연속적인 숫자) / 분류용 정답 (0 또는 1)
target_reg = torch.randn(100, 1)
target_bin = torch.randint(0, 2, (100, 1)).float()

# 2. 모델 및 손실 함수 설정
model_reg = ComparisonNet(output_size=1)
model_bin = ComparisonNet(output_size=1)

criterion_mse = nn.MSELoss()            # [개념: MSE] 회귀용 오차 계산
criterion_bce = nn.BCEWithLogitsLoss()  # [개념: BCEWithLogits] 이진 분류용 (Sigmoid 포함)

# 3. 최적화 도구 (경사하강법 기반 Adam)
optimizer_reg = optim.Adam(model_reg.parameters(), lr=0.01)
optimizer_bin = optim.Adam(model_bin.parameters(), lr=0.01)

# 4. 학습 및 오차 비교
print(f"{'Epoch':<8} | {'Regression (MSE)':<20} | {'Binary (BCEWithLogits)':<20}")
print("-" * 60)

for epoch in range(101):
    # --- Regression 학습 ---
    optimizer_reg.zero_grad()
    out_reg = model_reg(inputs)
    loss_reg = criterion_mse(out_reg, target_reg) # 오차 구하기
    loss_reg.backward()                           # 역전파
    optimizer_reg.step()                          # 경사하강법 업데이트

    # --- Binary Classification 학습 ---
    optimizer_bin.zero_grad()
    out_bin = model_bin(inputs)
    loss_bin = criterion_bce(out_bin, target_bin) # 오차 구하기
    loss_bin.backward()                           # 역전파
    optimizer_bin.step()                          # 경사하강법 업데이트

    if epoch % 20 == 0:
        print(f"{epoch:<8} | {loss_reg.item():<20.4f} | {loss_bin.item():<20.4f}")

Epoch    | Regression (MSE)     | Binary (BCEWithLogits)
------------------------------------------------------------
0        | 1.1476               | 0.7397              
20       | 0.6271               | 0.6008              
40       | 0.3648               | 0.4989              
60       | 0.2096               | 0.3931              
80       | 0.1094               | 0.2998              
100      | 0.0446               | 0.2142              
